In [ ]:
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib import rcParams


%matplotlib inline

# Opening JSON file
g = open('../data/genres.json')
 
# returns JSON object as a dictionary
genre_json = json.load(g)

# since the value lists have varying lengths, we have to use a lambda function to iterate over the keys and values as a series
genres = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in genre_json.items()]))

genres = genres.T

#genres = pd.melt(genres)

#genres = genres.reset_index(drop=True)
#genres.columns = ['artist', 'genre']

genres.head()

In [ ]:
genres_fix = genres[0].str.split(',', expand = True)
genres_fix.head()

In [ ]:
genres = genres.drop(0, axis = 1)
genres.head()

In [ ]:
genres_final = pd.merge(genres, genres_fix, left_index = True, right_index = True, how = 'outer')
pd.set_option('max_columns', None)
genres_final.head()

In [ ]:
genres_final['1_y'].head()

In [ ]:
genres_finalx2 = genres_final.T
genres_finalx2.head()

In [ ]:
genres_melt = pd.melt(genres_finalx2)
genres_melt.head(60)

In [ ]:
genres_melt = genres_melt.dropna()

In [ ]:
genres_melt.drop_duplicates()

In [ ]:
genres_melt.columns = ['Recording Artist', 'Genre']
genres_melt.head(30)

In [ ]:
#Making genre column all capps

genres_melt['Genre'] = genres_melt['Genre'].str.upper()
genres_melt.head(60)

In [ ]:
#Exploring dataset
genres_melt.tail(20)

In [ ]:
genres_melt.Genre.value_counts()

In [ ]:
genres_melt['Genre'].str.strip()

In [ ]:
mlc = pd.read_csv('../data/MLC_Data_Sample.csv')
mlc_df = pd.DataFrame(mlc)
mlc_df.head()

In [ ]:
mlc_df.info()

In [ ]:
mlc_df = mlc_df.drop_duplicates()
mlc_df.info()

In [ ]:
#drop NA for MLC DF
mlc_df = mlc_df.dropna()

In [ ]:
genres_df = pd.DataFrame(genres_melt)

In [ ]:
mlc_and_genres_df = pd.merge(genres_df, mlc_df, on=["Recording Artist"], how='left')
mlc_and_genres_df.head()

In [ ]:
mlc_and_genres_df.info()

In [ ]:
mlc_and_genres_df.drop_duplicates()

In [ ]:
mlc_and_genres_df.dropna()

In [ ]:
royalties = pd.read_csv('../data/royalties_per_platform.csv')
royalties.head(10)                        

In [ ]:
royalties = royalties.drop(columns=['source'])
royalties.head(10)

In [ ]:
royalties_df = pd.DataFrame(royalties)
royalties_df.head(10)

In [ ]:
royalties_df.rename(columns = {'platform':'Streaming Platform (DSP)', 'royalties_per_stream':'Royalties per Stream'}, inplace = True)

In [ ]:
royalties_df.head()

In [ ]:
mlcGenresRoyalties_df = pd.merge(mlc_and_genres_df, royalties_df, on=['Streaming Platform (DSP)'], how='left')


In [ ]:
mlcGenresRoyalties_df.head()

In [ ]:
mlcGenresRoyalties_df["Streams"] = mlcGenresRoyalties_df["Streams"].str.replace(",","")
mlcGenresRoyalties_df.head()

In [ ]:
duplicates = mlcGenresRoyalties_df[mlcGenresRoyalties_df.duplicated()]
duplicates

In [ ]:
mlcGenresRoyalties_df = mlcGenresRoyalties_df.drop_duplicates()

In [ ]:
new_duplicates = mlcGenresRoyalties_df[mlcGenresRoyalties_df.duplicated()]
new_duplicates

In [ ]:
nulls = mlcGenresRoyalties_df.isnull().sum()
nulls

In [ ]:
mlcGenresRoyalties_df = mlcGenresRoyalties_df.dropna()
new_nulls = mlcGenresRoyalties_df.isnull().sum()
new_nulls

In [ ]:
mlcGenresRoyalties_df['Streams'] = pd.to_numeric(mlcGenresRoyalties_df['Streams'])

In [ ]:
mlcGenresRoyalties_df.info()

In [ ]:
mlcGenresRoyalties_df.head()

In [ ]:
mlcGenresRoyalties_df['Royalties per Stream'] = mlcGenresRoyalties_df['Royalties per Stream'].str.replace("$","")
mlcGenresRoyalties_df['Royalties per Stream'] = pd.to_numeric(mlcGenresRoyalties_df['Royalties per Stream'])

In [ ]:
mlcGenresRoyalties_df.info()

In [ ]:
#mlcGenresRoyalties_df = mlcGenresRoyalties_df.head(10)
commonGenres = mlcGenresRoyalties_df['Genre'].value_counts()
commonGenres_df = pd.DataFrame(commonGenres).reset_index()
commonGenres2 = commonGenres_df.head(10)
commonGenres2

In [ ]:
rcParams['figure.figsize'] = 15,8
sns.set(font_scale = 2)
commonGenres2 = sns.barplot(x="Genre", y="index", data=commonGenres2,
                 palette="YlGnBu_r")
commonGenres2.set(ylabel= None)
commonGenres2.set(xlabel="Genre Count")
commonGenres2.set_title('Ten Most Common Genres in MLC Matching Sample');




In [ ]:
mlcGenresRoyalties_df.info()

In [ ]:
#Number of Streams per Genre
#avgStreamsPerGenre = mlcGenresRoyalties_df.groupby('Genre')[['Streams']].mean()
#avgStreamsPerGenreDesc = avgStreamsPerGenre.sort_values(['Streams'], ascending=False)
#avgStreamsPerGenreDesc


genre_streams_avg = mlcGenresRoyalties_df.groupby('Genre', as_index=False)[['Streams']].mean().sort_values(['Streams'],ascending=False)
genre_streams_avg = genre_streams_avg.head(10)
genre_streams_avg


In [ ]:
rcParams['figure.figsize'] = 15,8
sns.set(font_scale = 2)
genreStreamsAvgChart = sns.barplot(x="Streams", y="Genre", data=genre_streams_avg,
                 palette="YlGnBu_r")
genreStreamsAvgChart.set(ylabel=None)
genreStreamsAvgChart.set_title('Top 10 Average Streams Per Genre');

